In [1]:
!python3 -m pip install --upgrade pip

In [2]:
!pip3 install arxiv==2.1.0
!pip3 install python-dotenv tiktoken
# !pip install openai==0.27.8
# !pip install openai==1.2.3
!pip install openai==1.3.4
!pip install -U duckduckgo-search==4.4

In [3]:
import os
import json
import datetime as dt
import yaml
import warnings


import arxiv
import openai
from openai import OpenAI
from dotenv import load_dotenv

from duckduckgo_search import DDGS, AsyncDDGS
import asyncio

# すべての警告を無視する
warnings.filterwarnings('ignore')

In [4]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [5]:
class StreamingLLMMemory:
    """
    StreamingLLMMemory クラスは、最新のメッセージと特定数のattention sinksを
    メモリに保持するためのクラスです。
    
    attention sinksは、言語モデルが常に注意を向けるべき初期のトークンで、
    モデルが過去の情報を"覚えて"いるのを手助けします。
    """
    def __init__(self, max_length=10, attention_sinks=4):
        """
        メモリの最大長と保持するattention sinksの数を設定
        
        :param max_length: int, メモリが保持するメッセージの最大数
        :param attention_sinks: int, 常にメモリに保持される初期トークンの数
        """
        self.memory = []
        self.max_length = max_length
        self.attention_sinks = attention_sinks
    
    def get(self):
        """
        現在のメモリの内容を返します。
        
        :return: list, メモリに保持されているメッセージ
        """
        return self.memory
    
    def add(self, message):
        """
        新しいメッセージをメモリに追加し、メモリがmax_lengthを超えないように
        調整します。もしmax_lengthを超える場合、attention_sinksと最新のメッセージを
        保持します。
        
        :param message: str, メモリに追加するメッセージ
        """
        self.memory.append(message)
        if len(self.memory) > self.max_length:
            self.memory = self.memory[:self.attention_sinks] + self.memory[-(self.max_length-self.attention_sinks):]
    
    def add_pair(self, user_message, ai_message):
        """
        ユーザーとAIからのメッセージのペアをメモリに追加します。
        
        :param user_message: str, ユーザーからのメッセージ
        :param ai_message: str, AIからのメッセージ
        """
        # self.add("User: " + user_message)
        # self.add("AI: " + ai_message)
        self.add({"role": "user", "content": user_message})
        self.add({"role": "assistant", "content": ai_message})
    
    # ここにはStreamingLLMとのインタラクションのための追加のメソッドを
    # 実装することもできます。例えば、generate_response, update_llm_modelなどです。

In [6]:
load_dotenv()

True

In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
# MODEL_NAME = "gpt-4-0125-preview"
TEMPERATURE = 0.7
# OpenAIクライアントの初期化
client = OpenAI()

In [9]:
# 質問
# question = "禁闕の変について教えてください"
# question = "今日の東京と稚内市の天気を教えてください"

question = """最近の世界経済動向を分析し、これから市場で注目すべき3つの産業を予測してください。それぞれの産業が注目される理由も含めて説明してください。
また、将来発展するような市場についても教えてください。"""

# 過去の失敗結果のログ
research_res_1 = """最新のPwC調査レポートによると、2042年までに世界経済の規模は倍増し、中国が購買力平価（PPP）ベースで米国を抜き世界最大の経済大国になると予測されています。また、市場為替レート（MER）ベースでも2030年までに世界最大となる見込みです。2050年にはインドが米国を抜き世界第2位、インドネシアが第4位の経済大国となり、日本やドイツなどの先進国を抜くとされています。さらに、2050年までに主要経済大国7カ国のうち6カ国が新興国になると予測されています。ベトナムは2050年までに世界で最も高成長を遂げる経済大国となり、予測GDPの世界順位は第20位に上昇する見込みです。EU加盟27カ国が世界GDPに占める割合は2050年までに10％未満に低下し、世界経済成長率のベースライン予測は2022年の3.5％から2023年は3.0％、2024年は2.9％へ鈍化すると見込まれています。先進国の成長率は、政策の引き締めの影響が出始め、2022年の2.6％から、2023年は1.5％、2024年は1.4％へ鈍化する見込みです。新興市場国と発展途上国の成長率はやや鈍化し、2022年の4.1％から、2023年と2024年はともに4.0％となる見込みです。世界のインフレ率は、国際的な一次産品価格の下落が金融政策の引き締めと合わさり、2022年の8.7％から2023年は6.9％、2024年は5.8％へと安定的に鈍化する見込みです。デジタル変革が進む中で、デジタル技術とグローバルなデータフローの指数関数的な発展・成長が経済のルールを書き換えつつあり、第四次産業革命が進展しています。これにより、大量生産・画一的サービス提供から個々にカスタマイズされた生産・サービスの提供、既に存在している資源・資産の効率的な活用、AIやロボットによる、従来人間によって行われていた労働の補助・代替などが可能となっています。
"""
research_res_2 = """この検索結果では、世界経済の成長率予測に関する日本語のPDF版の概要が提供されています。経済部門間や地域間の格差拡大の中で、世界経済の回復が減速している状況が説明されています。2022年の推計成長率は3.5%から、2023年と2024年はそれぞれ3.0%に減速する見込みです。また、インフレへの対処として中央銀行が政策金利を引き上げ続けていることが、経済活動の重しとなっています。世界の総合インフレ率は、2022年の8.7%から2023年は6.8%、2024年は5.2%へと鈍化する見込みです。さらに、金融政策の引き締めや制約的な金融環境、貿易や投資の世界的な低迷により、成長率のさらなる減速が見込まれています。下振れリスクとしては、中東紛争の激化、金融ストレス、根強いインフレ、貿易の分断化、気候関連の自然災害などが挙げられています。新興国及び途上国では、財政政策の変動が大きく、適正なマクロ経済・構造政策と機能する制度が、投資と長期的な成長の促進に不可欠とされています。また、2024年の世界経済の成長率は2.4%と予想され、3年連続の減速が見込まれています。先進国と新興市場国、発展途上国の成長率の見通しも含まれており、先進国は政策引き締めの影響で、新興市場国と発展途上国はやや鈍化する見込みです。米欧の高金利・高インフレ、中国の景気減速の下で、世界経済は2023年末から2024年前半にかけて停滞感が強まる展開が予想されています。さらに、戦争が主な要因で一次産品が値上がりし、物価圧力が広範囲に広がっている状況や、IMFによる世界のGDP成長率予測の下方修正なども報告されています。
"""

## ICRL

In [10]:
# 4件のメッセージを記憶するように設定
search_memory = StreamingLLMMemory(max_length=4)

In [11]:
search_memory.add(research_res_1)

In [12]:
search_memory.add(research_res_2)

In [13]:
get_memory = search_memory.get()
# get_memory

In [14]:
memory_str = ""
for g_m in get_memory:
    memory_str += g_m
print(memory_str)

最新のPwC調査レポートによると、2042年までに世界経済の規模は倍増し、中国が購買力平価（PPP）ベースで米国を抜き世界最大の経済大国になると予測されています。また、市場為替レート（MER）ベースでも2030年までに世界最大となる見込みです。2050年にはインドが米国を抜き世界第2位、インドネシアが第4位の経済大国となり、日本やドイツなどの先進国を抜くとされています。さらに、2050年までに主要経済大国7カ国のうち6カ国が新興国になると予測されています。ベトナムは2050年までに世界で最も高成長を遂げる経済大国となり、予測GDPの世界順位は第20位に上昇する見込みです。EU加盟27カ国が世界GDPに占める割合は2050年までに10％未満に低下し、世界経済成長率のベースライン予測は2022年の3.5％から2023年は3.0％、2024年は2.9％へ鈍化すると見込まれています。先進国の成長率は、政策の引き締めの影響が出始め、2022年の2.6％から、2023年は1.5％、2024年は1.4％へ鈍化する見込みです。新興市場国と発展途上国の成長率はやや鈍化し、2022年の4.1％から、2023年と2024年はともに4.0％となる見込みです。世界のインフレ率は、国際的な一次産品価格の下落が金融政策の引き締めと合わさり、2022年の8.7％から2023年は6.9％、2024年は5.8％へと安定的に鈍化する見込みです。デジタル変革が進む中で、デジタル技術とグローバルなデータフローの指数関数的な発展・成長が経済のルールを書き換えつつあり、第四次産業革命が進展しています。これにより、大量生産・画一的サービス提供から個々にカスタマイズされた生産・サービスの提供、既に存在している資源・資産の効率的な活用、AIやロボットによる、従来人間によって行われていた労働の補助・代替などが可能となっています。
この検索結果では、世界経済の成長率予測に関する日本語のPDF版の概要が提供されています。経済部門間や地域間の格差拡大の中で、世界経済の回復が減速している状況が説明されています。2022年の推計成長率は3.5%から、2023年と2024年はそれぞれ3.0%に減速する見込みです。また、インフレへの対処として中央銀行が政策金利を引き上げ続けていることが、経済活動の重しとなっています。世界の総合インフ

In [15]:
prompt = []
if memory_str != "":
    prompt.append({"role": "system", "content": f"The following text represents an inappropriate or failed response to a query, not aligning with the intended purpose. Please avoid generating such responses again.\n\n{memory_str}"})
prompt

[{'role': 'system',
  'content': 'The following text represents an inappropriate or failed response to a query, not aligning with the intended purpose. Please avoid generating such responses again.\n\n最新のPwC調査レポートによると、2042年までに世界経済の規模は倍増し、中国が購買力平価（PPP）ベースで米国を抜き世界最大の経済大国になると予測されています。また、市場為替レート（MER）ベースでも2030年までに世界最大となる見込みです。2050年にはインドが米国を抜き世界第2位、インドネシアが第4位の経済大国となり、日本やドイツなどの先進国を抜くとされています。さらに、2050年までに主要経済大国7カ国のうち6カ国が新興国になると予測されています。ベトナムは2050年までに世界で最も高成長を遂げる経済大国となり、予測GDPの世界順位は第20位に上昇する見込みです。EU加盟27カ国が世界GDPに占める割合は2050年までに10％未満に低下し、世界経済成長率のベースライン予測は2022年の3.5％から2023年は3.0％、2024年は2.9％へ鈍化すると見込まれています。先進国の成長率は、政策の引き締めの影響が出始め、2022年の2.6％から、2023年は1.5％、2024年は1.4％へ鈍化する見込みです。新興市場国と発展途上国の成長率はやや鈍化し、2022年の4.1％から、2023年と2024年はともに4.0％となる見込みです。世界のインフレ率は、国際的な一次産品価格の下落が金融政策の引き締めと合わさり、2022年の8.7％から2023年は6.9％、2024年は5.8％へと安定的に鈍化する見込みです。デジタル変革が進む中で、デジタル技術とグローバルなデータフローの指数関数的な発展・成長が経済のルールを書き換えつつあり、第四次産業革命が進展しています。これにより、大量生産・画一的サービス提供から個々にカスタマイズされた生産・サービスの提供、既に存在している資源・資産の効率的な活用、AIやロボットによる、従来人間によって行われていた労働の補助・代替などが可能となっていま

In [16]:
# 4件のメッセージを記憶するように設定
search_memory = StreamingLLMMemory(max_length=4)

In [17]:
def ICRL(memory, research_res = ""):
    memory.add(research_res)
    get_memory = memory.get()
    memory_str = ""
    for g_m in get_memory:
        memory_str += g_m
    
    prompt = []
    if memory_str != "":
        prompt.append({"role": "system", "content": f"The following text represents an inappropriate or failed response to a query, not aligning with the intended purpose. Please avoid generating such responses again.\n\n{memory_str}"})
    return prompt, memory

In [18]:
prompt, search_memory = ICRL(search_memory)
print(prompt)

[]


In [19]:
if prompt == []:
    print("OK")
else:
    print(prompt)

OK


In [20]:
research_res = "世界経済の成長率は、金融政策の引き締め、制約的な金融環境、貿易や投資の低迷により、本年さらに減速すると予測されています。下振れリスクには、中東紛争の激化、金融ストレス、インフレ、貿易の分断化、気候関連の自然災害などが挙げられます。経済成長を維持するためには、債務免除、貿易統合の促進、気候変動対策、食料不安の軽減などのための国際協調が必要です。新興国や途上国では、財政政策の変動が大きく、特に一次産品を輸出する国々において景気循環増幅的な影響が見られます。これらの国々では、適切なマクロ経済政策と機能する制度が投資と長期的な成長を支えるために不可欠です。2024年の世界経済成長率は2.4％と予測され、3年連続での減速が見込まれています。しかし、2024年の成長率予測は前の予測よりもやや改善しており、ソフトランディングの可能性が見えてきています。中央銀行による高い政策金利や財政支援の縮小が経済活動の重しとなりつつも、2024年と2025年の成長率予測は歴史的平均を下回るものの、いくつかの国での予想以上の底堅さが支えになっています。インフレは、金融政策の引き締めと国際的な一次産品価格の下落により安定的に鈍化する見込みです。世界銀行の予測では、2021年から2023年にかけての世界経済の成長率は大きく鈍化し、新興国や途上国の成長率も減速すると予測されています。IMFによると、2022年の世界経済の成長率予測は、ロシアによるウクライナ侵攻やインフレなどの影響を考慮して下方修正されています。世界銀行は、2023年の世界経済成長率が大幅に減速すると予測しており、グローバルリスク報告書では、中短期及び長期の潜在リスクが分析されています。"

In [21]:
prompt, search_memory = ICRL(search_memory, research_res)

In [22]:
if prompt == []:
    print("OK")
else:
    print(prompt)

[{'role': 'system', 'content': 'The following text represents an inappropriate or failed response to a query, not aligning with the intended purpose. Please avoid generating such responses again.\n\n世界経済の成長率は、金融政策の引き締め、制約的な金融環境、貿易や投資の低迷により、本年さらに減速すると予測されています。下振れリスクには、中東紛争の激化、金融ストレス、インフレ、貿易の分断化、気候関連の自然災害などが挙げられます。経済成長を維持するためには、債務免除、貿易統合の促進、気候変動対策、食料不安の軽減などのための国際協調が必要です。新興国や途上国では、財政政策の変動が大きく、特に一次産品を輸出する国々において景気循環増幅的な影響が見られます。これらの国々では、適切なマクロ経済政策と機能する制度が投資と長期的な成長を支えるために不可欠です。2024年の世界経済成長率は2.4％と予測され、3年連続での減速が見込まれています。しかし、2024年の成長率予測は前の予測よりもやや改善しており、ソフトランディングの可能性が見えてきています。中央銀行による高い政策金利や財政支援の縮小が経済活動の重しとなりつつも、2024年と2025年の成長率予測は歴史的平均を下回るものの、いくつかの国での予想以上の底堅さが支えになっています。インフレは、金融政策の引き締めと国際的な一次産品価格の下落により安定的に鈍化する見込みです。世界銀行の予測では、2021年から2023年にかけての世界経済の成長率は大きく鈍化し、新興国や途上国の成長率も減速すると予測されています。IMFによると、2022年の世界経済の成長率予測は、ロシアによるウクライナ侵攻やインフレなどの影響を考慮して下方修正されています。世界銀行は、2023年の世界経済成長率が大幅に減速すると予測しており、グローバルリスク報告書では、中短期及び長期の潜在リスクが分析されています。'}]


In [23]:
# 再確認
prompt, search_memory = ICRL(search_memory, research_res)
print(prompt)

[{'role': 'system', 'content': 'The following text represents an inappropriate or failed response to a query, not aligning with the intended purpose. Please avoid generating such responses again.\n\n世界経済の成長率は、金融政策の引き締め、制約的な金融環境、貿易や投資の低迷により、本年さらに減速すると予測されています。下振れリスクには、中東紛争の激化、金融ストレス、インフレ、貿易の分断化、気候関連の自然災害などが挙げられます。経済成長を維持するためには、債務免除、貿易統合の促進、気候変動対策、食料不安の軽減などのための国際協調が必要です。新興国や途上国では、財政政策の変動が大きく、特に一次産品を輸出する国々において景気循環増幅的な影響が見られます。これらの国々では、適切なマクロ経済政策と機能する制度が投資と長期的な成長を支えるために不可欠です。2024年の世界経済成長率は2.4％と予測され、3年連続での減速が見込まれています。しかし、2024年の成長率予測は前の予測よりもやや改善しており、ソフトランディングの可能性が見えてきています。中央銀行による高い政策金利や財政支援の縮小が経済活動の重しとなりつつも、2024年と2025年の成長率予測は歴史的平均を下回るものの、いくつかの国での予想以上の底堅さが支えになっています。インフレは、金融政策の引き締めと国際的な一次産品価格の下落により安定的に鈍化する見込みです。世界銀行の予測では、2021年から2023年にかけての世界経済の成長率は大きく鈍化し、新興国や途上国の成長率も減速すると予測されています。IMFによると、2022年の世界経済の成長率予測は、ロシアによるウクライナ侵攻やインフレなどの影響を考慮して下方修正されています。世界銀行は、2023年の世界経済成長率が大幅に減速すると予測しており、グローバルリスク報告書では、中短期及び長期の潜在リスクが分析されています。世界経済の成長率は、金融政策の引き締め、制約的な金融環境、貿易や投資の低迷により、本年さらに減速すると予測されています。下振れリスクには、中東紛争の激化、金融スト

In [46]:
prompt[0]

{'role': 'system',
 'content': 'The following text represents an inappropriate or failed response to a query, not aligning with the intended purpose. Please avoid generating such responses again.\n\n世界経済の成長率は、金融政策の引き締め、制約的な金融環境、貿易や投資の低迷により、本年さらに減速すると予測されています。下振れリスクには、中東紛争の激化、金融ストレス、インフレ、貿易の分断化、気候関連の自然災害などが挙げられます。経済成長を維持するためには、債務免除、貿易統合の促進、気候変動対策、食料不安の軽減などのための国際協調が必要です。新興国や途上国では、財政政策の変動が大きく、特に一次産品を輸出する国々において景気循環増幅的な影響が見られます。これらの国々では、適切なマクロ経済政策と機能する制度が投資と長期的な成長を支えるために不可欠です。2024年の世界経済成長率は2.4％と予測され、3年連続での減速が見込まれています。しかし、2024年の成長率予測は前の予測よりもやや改善しており、ソフトランディングの可能性が見えてきています。中央銀行による高い政策金利や財政支援の縮小が経済活動の重しとなりつつも、2024年と2025年の成長率予測は歴史的平均を下回るものの、いくつかの国での予想以上の底堅さが支えになっています。インフレは、金融政策の引き締めと国際的な一次産品価格の下落により安定的に鈍化する見込みです。世界銀行の予測では、2021年から2023年にかけての世界経済の成長率は大きく鈍化し、新興国や途上国の成長率も減速すると予測されています。IMFによると、2022年の世界経済の成長率予測は、ロシアによるウクライナ侵攻やインフレなどの影響を考慮して下方修正されています。世界銀行は、2023年の世界経済成長率が大幅に減速すると予測しており、グローバルリスク報告書では、中短期及び長期の潜在リスクが分析されています。世界経済の成長率は、金融政策の引き締め、制約的な金融環境、貿易や投資の低迷により、本年さらに減速すると予測されています。下振れリスクには、中東紛争の激化、金融スト

## ICSL

質問に対して、対応するプロンプトを選択する
あなたの関心ごとは、
- 自然科学（物理学、化学、生物学）
- 社会科学（心理学、経済学、政治学）
- 人文科学（歴史、文学、哲学）
- 技術と工学（コンピュータサイエンス、工学、医療技術）
- 映画とテレビ
- 音楽
- ゲームとインタラクティブメディア
- 趣味とインタレスト（例：料理、旅行、ファッション）
- 健康とウェルネス
- フィットネスとエクササイズ
- 栄養と食事
- メンタルヘルス
- 企業情報と業界分析
- 経済動向と市場予測
- 起業とビジネス戦略
- 金融と投資
- 新技術とイノベーション
- ITとコンピューター技術
- 環境科学と持続可能性
- 宇宙と天文学
- 教育資源とオンラインコース
- キャリア開発と就職活動
- スキル向上と自己啓発
- 学習ツールとリソース

についてです


In [25]:
{"role": "system", "content": ""}

{'role': 'system', 'content': ''}

In [27]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


In [31]:
word = "自然科学（物理学、化学、生物学）"
embedding = get_embedding(word, model='text-embedding-3-small')
embedding

[0.029698140919208527,
 0.03089897148311138,
 0.00651495112106204,
 0.04527308791875839,
 0.0461692288517952,
 -0.00841477233916521,
 -0.03367701172828674,
 -0.03054051473736763,
 0.005081124138087034,
 0.005538156721740961,
 -0.002314286306500435,
 0.019410433247685432,
 -0.034178849309682846,
 -0.00918545387685299,
 0.026920100674033165,
 0.02824639156460762,
 0.00017810819554142654,
 0.03270917758345604,
 0.029500989243388176,
 -0.024769360199570656,
 0.0032149089965969324,
 -0.03803225979208946,
 0.0005472066113725305,
 -0.05391189455986023,
 -0.028604848310351372,
 -0.0467069149017334,
 0.05570417642593384,
 0.0031252948101609945,
 0.011336194351315498,
 0.003140977118164301,
 0.02614942006766796,
 -0.03355155140161514,
 0.0009885564213618636,
 0.0025786482729017735,
 -0.02539665997028351,
 0.05090085789561272,
 -0.019715121015906334,
 -0.04057730361819267,
 -0.006075841840356588,
 -0.031669653952121735,
 0.0235506072640419,
 -1.1359297786839306e-05,
 -0.018146872520446777,
 0.004

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [33]:
# 単語リスト
words = [
    "自然科学（物理学、化学、生物学）", "社会科学（心理学、経済学、政治学）", "人文科学（歴史、文学、哲学）",
    "技術と工学（コンピュータサイエンス、工学、医療技術）", "映画とテレビ", "音楽", "ゲームとインタラクティブメディア",
    "趣味とインタレスト（例：料理、旅行、ファッション）", "健康とウェルネス", "フィットネスとエクササイズ",
    "栄養と食事", "メンタルヘルス", "企業情報と業界分析", "経済動向と市場予測", "起業とビジネス戦略",
    "金融と投資", "新技術とイノベーション", "ITとコンピューター技術", "環境科学と持続可能性",
    "宇宙と天文学", "教育資源とオンラインコース", "キャリア開発と就職活動", "スキル向上と自己啓発", "学習ツールとリソース"
]


# 各単語の埋め込みベクトルを格納するDataFrameを初期化
df_embeddings = pd.DataFrame()

embeddings = []
for word in words:
    embedding = get_embedding(word, model='text-embedding-3-small')
    embeddings.append({'word': word, 'embedding': embedding})

# 辞書のリストからDataFrameを作成
df_embeddings = pd.DataFrame(embeddings)

# DataFrameをCSVファイルに保存
df_embeddings.to_csv('./data/embedded_words.csv', index=False)

df_embeddings.head()

,word,embedding
0,自然科学（物理学、化学、生物学）,"[0.029698140919208527, 0.03089897148311138, 0...."
1,社会科学（心理学、経済学、政治学）,"[0.005710966885089874, -0.013056189753115177, ..."
2,人文科学（歴史、文学、哲学）,"[0.040362611413002014, 0.012367785908281803, 0..."
3,技術と工学（コンピュータサイエンス、工学、医療技術）,"[-0.008590849116444588, 0.009956416673958302, ..."
4,映画とテレビ,"[0.031254928559064865, 0.036112505942583084, -..."


In [43]:
question = """最近の世界経済動向を分析し、これから市場で注目すべき3つの産業を予測してください。それぞれの産業が注目される理由も含めて説明してください。
また、将来発展するような市場についても教えてください。"""

embedding_df = pd.read_csv('./data/embedded_words.csv')
embedding_df.head()

# 質問文の埋め込みベクトルを取得
question_embedding = get_embedding(question, model='text-embedding-3-small')

# 文字列形式の埋め込みベクトルをnumpy配列に変換
embedding_df['embedding'] = embedding_df['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=','))

# 類似度計算のために、質問文の埋め込みベクトルと単語の埋め込みベクトルを配列に格納
word_embeddings = np.array(embedding_df['embedding'].tolist())

# コサイン類似度を計算
similarities = cosine_similarity([question_embedding], word_embeddings)

# 最も類似度が高い単語を抽出
most_similar_word_index = np.argmax(similarities)
most_similar_word = embedding_df.iloc[most_similar_word_index]['word']

print(f"質問に最も類似している単語は: {most_similar_word}")

In [44]:
def ICSL_search_system(question, embedding_df):
    # 質問文の埋め込みベクトルを取得
    question_embedding = get_embedding(question, model='text-embedding-3-small')
    
    # 文字列形式の埋め込みベクトルをnumpy配列に変換
    embedding_df['embedding'] = embedding_df['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=','))

    # 類似度計算のために、質問文の埋め込みベクトルと単語の埋め込みベクトルを配列に格納
    word_embeddings = np.array(embedding_df['embedding'].tolist())

    # コサイン類似度を計算
    similarities = cosine_similarity([question_embedding], word_embeddings)

    # 最も類似度が高い単語を抽出
    most_similar_word_index = np.argmax(similarities)
    most_similar_word = embedding_df.iloc[most_similar_word_index]['word']
    
    return most_similar_word

In [45]:
question = """最近の世界経済動向を分析し、これから市場で注目すべき3つの産業を予測してください。それぞれの産業が注目される理由も含めて説明してください。
また、将来発展するような市場についても教えてください。"""

embedding_df = pd.read_csv('./data/embedded_words.csv')

# テスト
most_similar_word = ICSL_search_system(question, embedding_df)
most_similar_word

'経済動向と市場予測'